In [18]:
# sample dataset
# Format: each row is a data point
# The last column is the label 
# The first two columns are features

trainingData = [['Green',3,'Mango'],
               ['Yellow',3,'Mango'],
               ['Red',1,'Grape'],
               ['Red',1,'Grape'],
               ['Yellow',3,'Lemon']]
#column Labels
header = ['color','diameter','label']


def unique_vals(rows,col):
    """find the unique values in the columns in the dataset"""
    return set([row[col]for row in rows])
####### 
#unique_vals(trainingData, 0)
#unique_vals(trainingData, 1)

def class_count(rows):
    """counts the number of each type of example in a dataset"""
    counts = {} # a dictionary of label -count
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# class_counts(trainingData)

#isinstance is a function used to check whether a particular value is of a particular type returns true or false
def is_numeric(value):
    """Tests if a value is numeric"""
    return isinstance(value,int) or isinstance(value,float)
#is_numeric(7)

In [16]:

set([1,2,2,3,4,5,5,5,6,7,7,7,8,9])

things = (1,3,4,5,6,7,78,8,8,8,8)
for i in range(20):
    if i not in things:
        print(i)
    

0
2
9
10
11
12
13
14
15
16
17
18
19


In [4]:
class Question:
    """used to partition a dataset.
    This class just records a 'column number '(e.g, 0 for color) and a column value(e.g,green).
    the 'match method' is used to compare the feature value in an example to the feature value 
    stored in the question. See the demo below"""
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match (self,example):
        #compare the feature value is in an example to the feature value in question
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else: 
            return val == self.value
    def __repr__(self):
        #this is just a helper method to print question in readable format
        condition = '=='
        if is_numeric(self.value):
            return "Is %s %s %s?" % (
            header[self.column],condition,str(self.value))
        
def partition(rows, question):
    """partitions a dataset 
    
    For each row in the dataset, check if it matches the question. If so add it to 'true rows',
    otherwise add it to 'false row'"""
        true_rows , false_rows = [],[]
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows
# Demo    
# Let's partition the training data based on whether rows are red.
# true_rows, false_rows = partition(training_data,Question(0,'Red'))
#this will contain all the red rows(true_rows)
# false_rows will contain everything else


def gini(rows):
    """Calculates the Gini index(level of impurity in each row)"""
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
        return impurity
    
# demo:
# let's look at the same example to understand how gini impurity works
        
    
def info_gain(left,right, current_uncertainty):
    """information Gain.
    
    The uncertainty of the starting node, minus the weighted impurity of
    two child nods
    """
    p = float(len(left))/(len(left)) + len(right)
    
    return current_uncertainty - p*gini(left) - (1-p)*gini(right)

####
# demo 
# Calculate the uncertainty of our training data
# current_uncertainty - gini(training_data)
#
# How much information do we gain by partioning on Green?
# true_rows,false_rows = partition(training_data,Question(0,'Green'))
# info_gain(true_rows, false_rows, current_uncertainty)
#
#What if we partitioned red instead
# true_rows, false_rows = partition(training_data,Question(0,'Red'))
# info_gain(true_rows, false_rows, current_uncertainty)

def find_best_split(rows):
    """Finds the best question to ask by iterating over every feature/value
    and calculating the information gain"""
    
    best_gain = 0 # to keep track of information gain
    best_question = None # keep tracks of the feature /value that produced it
    current_uncertainty = gini(rows)
    n_features = len(row[0])-1 # number of columns
    
    for col in range(n_features): #for each feature
        values = set([row[col]] for row in rows) # unique values in the column
        
        for val in values: # for each value
            
            question = Question(col, val)
            
            #try splitting the dataset
            true_rows, false_rows = partition(rows,question)
            
            #skip the split if it doesnt divide the dataset
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            # calculate the information gain from this split
            gain = info_gain(true_rows,false_rows,current_uncertainty)
            
            # you actually can use '>' instead of '>=' here
            # but i wan the tree to look a certain way for the dataset
            if gain >= best_gain:
                best_gain, best_question = gain, question
            return best_gain, best_question

    class Leaf:
        """A leaf node classifies the data.
        
        This holds a dictionarty of class (e.g,"mangeo")-> number of times
        it appears in the rows from the training data that reach this leaf.
        """
        def __init__(self, rows):
            self.predictions = class_counts(rows)
            
        
    class Decision_Node:
        """A decision node asks a question.
        This holds a reference to the question and to the two child nodes.
        """
        
        def __init__(self,
                    question,
                    true_branch,
                    false_branch):
            self.question = question
            self.true_branch = true_branch
            self.false_branch = false_branch
        def build_tree(rows):
            """Builds the tree"""
            # try partitioning the dataset on each of the unique attribute,
            # calclulte the information gain and return the question that produces the highest information gain
            gain, question = find_best_split(rows)
            
            # Base case: no further info gain
            # since we can ask no futher question we'll return a leaf
            
            if gain == 0: 
                return Leaf(rows)
            
            # if we reach here, we have found a useful feature/ value to partition on
            
            true_rows, false_rows = partition(rows, question)
            
            # recursively build the true branch
            true_branch = build_tree(true_rows)
            
            # recursively build the false branch
            false_branch = build_tree(false_rows)
            
            # return a question node. 
            # this records the best feature/value to ask at this point,
            # as well as the branches to follow
            # depending on the answer
            
            return Decision_Node(question,true_branch,false_branch)
        def print_tree (node , spacing=""):
            """Worlds most elegant tree printing function"""
            
            # Base case: we've reached a leaf
            if isinstance(node, Leaf):
                print(spacing + 'predict', node.predictions)
                return
            
            # print the question at this node
            print(spacing + str(node.question))
            
            # call this function recursively on the true branch
            print(spacing + '--> True:')
            print_tree(node.true_branch, spacing+" ")
            
            # call this function recursiverly on the false branch
            print(spacing + '--> False:')
            print_tree(node.false_branch, spacing+" ")
            
        def classify(row, node):
            
            # Base case: we've reached a leaf
            if isinstance(node,leaf):
                return node.predictions
            
            # decide whether to follow the true-branch of false-branch
            # compare the feature /value stored in the node to the example we're considering
            if node.question.match(row):
                return classify(row, node.true_branch)
            else:
                return classify(row, node.false_branch)
            
            
            
            
        
     
    
            
        
            

            

IndentationError: unexpected indent (<ipython-input-4-c5b26f119585>, line 29)

In [3]:
# __str__ easy to read for human consumption
# __repr__ to be un ambigious as possible to class objects

class car:
    def __init__(self,milage,color):
        self.milage = milage
        self.color = color
    def __repr__(self):
        return 'This car has a color of  {self.color} and the milage is {self.milage}'.format(self = self)
    
cars = car(1233,'yellow')
cars

This car has a color of  yellow and the milage is 1233